# Order Delivery Time Prediction

## Objectives
The objective of this assignment is to build a regression model that predicts the delivery time for orders placed through Porter. The model will use various features such as the items ordered, the restaurant location, the order protocol, and the availability of delivery partners.

The key goals are:
- Predict the delivery time for an order based on multiple input features
- Improve delivery time predictions to optimiae operational efficiency
- Understand the key factors influencing delivery time to enhance the model's accuracy

## Data Pipeline
The data pipeline for this assignment will involve the following steps:
1. **Data Loading**
2. **Data Preprocessing and Feature Engineering**
3. **Exploratory Data Analysis**
4. **Model Building**
5. **Model Inference**

## Data Understanding
The dataset contains information on orders placed through Porter, with the following columns:

| Field                     | Description                                                                                 |
|---------------------------|---------------------------------------------------------------------------------------------|
| market_id                 | Integer ID representing the market where the restaurant is located.                         |
| created_at                | Timestamp when the order was placed.                                                        |
| actual_delivery_time      | Timestamp when the order was delivered.                                                     |
| store_primary_category    | Category of the restaurant (e.g., fast food, dine-in).                                      |
| order_protocol            | Integer representing how the order was placed (e.g., via Porter, call to restaurant, etc.). |
| total_items               | Total number of items in the order.                                                         |
| subtotal                  | Final price of the order.                                                                   |
| num_distinct_items        | Number of distinct items in the order.                                                      |
| min_item_price            | Price of the cheapest item in the order.                                                    |
| max_item_price            | Price of the most expensive item in the order.                                              |
| total_onshift_dashers     | Number of delivery partners on duty when the order was placed.                              |
| total_busy_dashers        | Number of delivery partners already occupied with other orders.                             |
| total_outstanding_orders  | Number of orders pending fulfillment at the time of the order.                              |
| distance                  | Total distance from the restaurant to the customer.                                         |


## **Importing Necessary Libraries**

In [ ]:
# Import essential libraries for data manipulation and analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor


## **1. Loading the data**
Load 'porter_data_1.csv' as a DataFrame

In [ ]:
# Importing the file porter_data_1.csv

df = pd.read_csv('porter_data_1.csv')

## **2. Data Preprocessing and Feature Engineering** <font color = red>[15 marks]</font> <br>

#### **2.1 Fixing the Datatypes**  <font color = red>[5 marks]</font> <br>
The current timestamps are in object format and need conversion to datetime format for easier handling and intended functionality

##### **2.1.1** <font color = red>[2 marks]</font> <br>
Convert date and time fields to appropriate data type

In [ ]:
# Convert 'created_at' and 'actual_delivery_time' columns to datetime format

df['created_at'] = pd.to_datetime(df['created_at'])
df['actual_delivery_time'] = pd.to_datetime(df['actual_delivery_time'])

##### **2.1.2**  <font color = red>[3 marks]</font> <br>
Convert categorical fields to appropriate data type

In [ ]:
# Convert categorical features to category type
df['market_id'] = df['market_id'].astype('category')
df['store_primary_category'] = df['store_primary_category'].astype('category')
df['order_protocol'] = df['order_protocol'].astype('category')

#### **2.2 Feature Engineering** <font color = red>[5 marks]</font> <br>
Calculate the time taken to execute the delivery as well as extract the hour and day at which the order was placed

##### **2.2.1** <font color = red>[2 marks]</font> <br>
Calculate the time taken using the features `actual_delivery_time` and `created_at`

In [ ]:
# Calculate time taken in minutes
df['time_taken'] = (df['actual_delivery_time'] - df['created_at']).dt.total_seconds() / 60

##### **2.2.2** <font color = red>[3 marks]</font> <br>
Extract the hour at which the order was placed and which day of the week it was. Drop the unnecessary columns.

In [ ]:
# Extract the hour and day of week from the 'created_at' timestamp
df['order_hour'] = df['created_at'].dt.hour
df['day_of_week'] = df['created_at'].dt.dayofweek

# Create a categorical feature 'isWeekend'
df['isWeekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

In [ ]:
# Drop unnecessary columns
df = df.drop(['created_at', 'actual_delivery_time'], axis=1)

#### **2.3 Creating training and validation sets** <font color = red>[5 marks]</font> <br>

##### **2.3.1** <font color = red>[2 marks]</font> <br>
 Define target and input features

In [ ]:
# Define target variable (y) and features (X)
X = df.drop('time_taken', axis=1)
y = df['time_taken']

##### **2.3.2** <font color = red>[3 marks]</font> <br>
 Split the data into training and test sets

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## **3. Exploratory Data Analysis on Training Data** <font color = red>[20 marks]</font> <br>
1. Analyzing the correlation between variables to identify patterns and relationships
2. Identifying and addressing outliers to ensure the integrity of the analysis
3. Exploring the relationships between variables and examining the distribution of the data for better insights

#### **3.1 Feature Distributions** <font color = red> [7 marks]</font> <br>


In [ ]:
# Define numerical and categorical columns for easy EDA and data manipulation
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['category']).columns

##### **3.1.1** <font color = red>[3 marks]</font> <br>
Plot distributions for numerical columns in the training set to understand their spread and any skewness

In [ ]:
# Plot distributions for all numerical columns
plt.figure(figsize=(15, 10))
# Calculate how many rows we need based on number of columns
num_cols = len(numerical_cols)
rows = (num_cols + 2) // 3  # Calculate rows needed (3 columns per row)

for i, col in enumerate(numerical_cols):
    plt.subplot(rows, 3, i+1)  # Adjust grid size based on number of columns
    sns.histplot(X_train[col], kde=True)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

##### **3.1.2** <font color = red>[2 marks]</font> <br>
Check the distribution of categorical features

In [ ]:
# Distribution of categorical columns
plt.figure(figsize=(15, 5))
for i, col in enumerate(categorical_cols):
    plt.subplot(1, 3, i+1)
    sns.countplot(x=X_train[col])
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

##### **3.1.3** <font color = red>[2 mark]</font> <br>
Visualise the distribution of the target variable to understand its spread and any skewness

In [ ]:
# Distribution of time_taken
plt.figure(figsize=(10, 5))
sns.histplot(y_train, kde=True)
plt.title('Distribution of Delivery Time (minutes)')
plt.show()

#### **3.2 Relationships Between Features** <font color = red>[3 marks]</font> <br>

##### **3.2.1** <font color = red>[3 marks]</font> <br>
Scatter plots for important numerical and categorical features to observe how they relate to `time_taken`

In [ ]:
# Scatter plot to visualise the relationship between time_taken and other features
plt.figure(figsize=(15, 10))

# Get the number of numerical columns
n_cols = len(numerical_cols)

# Calculate rows and columns needed for the subplots
# Using ceil to round up for any fractional result
import math
n_rows = math.ceil(n_cols / 3)  # 3 columns per row

for i, col in enumerate(numerical_cols):
    plt.subplot(n_rows, 3, i+1)  # Dynamically calculate rows based on number of features
    sns.scatterplot(x=X_train[col], y=y_train)
    plt.title(f'{col} vs Delivery Time')
    
plt.tight_layout()
plt.show()

In [ ]:
# Show the distribution of time_taken for different hours
plt.figure(figsize=(10, 5))
sns.boxplot(x=X_train['order_hour'], y=y_train)
plt.title('Delivery Time by Hour of Day')
plt.show()

#### **3.3 Correlation Analysis** <font color = red>[5 marks]</font> <br>
Check correlations between numerical features to identify which variables are strongly related to `time_taken`

##### **3.3.1** <font color = red>[3 marks]</font> <br>
Plot a heatmap to display correlations

In [ ]:
# Plot the heatmap of the correlation matrix
corr_matrix = X_train[numerical_cols].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Numerical Features')
plt.show()

##### **3.3.2** <font color = red>[2 marks]</font> <br>
Drop the columns with weak correlations with the target variable

In [ ]:
# Drop 3-5 weakly correlated columns from training dataset
X_train = X_train.drop(['min_item_price', 'max_item_price', 'num_distinct_items'], axis=1)
X_test = X_test.drop(['min_item_price', 'max_item_price', 'num_distinct_items'], axis=1)

#### **3.4 Handling the Outliers** <font color = red>[5 marks]</font> <br>



##### **3.4.1** <font color = red>[2 marks]</font> <br>
Visualise potential outliers for the target variable and other numerical features using boxplots

In [ ]:
# Boxplot for time_taken
plt.figure(figsize=(10, 5))
sns.boxplot(y=y_train)
plt.title('Boxplot of Delivery Time')
plt.show()

##### **3.4.2** <font color = red>[3 marks]</font> <br>
Handle outliers present in all columns

In [ ]:
# Handle outliers
def remove_outliers(df, columns):
    for col in columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# Apply to training data
train_data = pd.concat([X_train, y_train], axis=1)
train_data = remove_outliers(train_data, numerical_cols)
X_train = train_data.drop('time_taken', axis=1)
y_train = train_data['time_taken']

## **4. Exploratory Data Analysis on Validation Data** <font color = red>[optional]</font> <br>
Optionally, perform EDA on test data to see if the distribution match with the training data

In [ ]:
# Define numerical and categorical columns for easy EDA and data manipulation
numerical_cols_test = X_test.select_dtypes(include=['int64', 'float64']).columns
categorical_cols_test = X_test.select_dtypes(include=['category']).columns

#### **4.1 Feature Distributions**


##### **4.1.1**
Plot distributions for numerical columns in the validation set to understand their spread and any skewness

In [ ]:
# Plot distributions for all numerical columns
plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_cols_test):
    plt.subplot(3, 3, i+1)
    sns.histplot(X_test[col], kde=True)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

##### **4.1.2**
Check the distribution of categorical features

In [ ]:
# Distribution of categorical columns
plt.figure(figsize=(15, 5))
for i, col in enumerate(categorical_cols_test):
    plt.subplot(1, 3, i+1)
    sns.countplot(x=X_test[col])
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

##### **4.1.3**
Visualise the distribution of the target variable to understand its spread and any skewness

In [ ]:
# Distribution of time_taken
plt.figure(figsize=(10, 5))
sns.histplot(y_test, kde=True)
plt.title('Distribution of Delivery Time (minutes)')
plt.show()

#### **4.2 Relationships Between Features**
Scatter plots for numerical features to observe how they relate to each other, especially to `time_taken`

In [ ]:
# Scatter plot to visualise the relationship between time_taken and other features
plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_cols_test):
    plt.subplot(3, 3, i+1)
    sns.scatterplot(x=X_test[col], y=y_test)
    plt.title(f'{col} vs Delivery Time')
plt.tight_layout()
plt.show()

#### **4.3** Drop the columns with weak correlations with the target variable

In [ ]:
# Drop the weakly correlated columns from training dataset
X_test = X_test.drop(['min_item_price', 'max_item_price', 'num_distinct_items'], axis=1)

## **5. Model Building** <font color = red>[15 marks]</font> <br>

#### **Import Necessary Libraries**

In [ ]:
# Import libraries
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

#### **5.1 Feature Scaling** <font color = red>[3 marks]</font> <br>

In [ ]:
# Apply scaling to the numerical columns
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test_scaled[numerical_cols] = scaler.transform(X_test[numerical_cols])

Note that linear regression is agnostic to feature scaling. However, with feature scaling, we get the coefficients to be somewhat on the same scale so that it becomes easier to compare them.

#### **5.2 Build a linear regression model** <font color = red>[5 marks]</font> <br>

You can choose from the libraries *statsmodels* and *scikit-learn* to build the model.

In [ ]:
# Create/Initialise the model
lr = LinearRegression()

In [ ]:
# Train the model using the training data
lr.fit(X_train_scaled, y_train)

In [ ]:
# Make predictions
y_pred = lr.predict(X_test_scaled)

In [ ]:
# Find results for evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Note that we have 12 (depending on how you select features) training features. However, not all of them would be useful. Let's say we want to take the most relevant 8 features.

We will use Recursive Feature Elimination (RFE) here.

For this, you can look at the coefficients / p-values of features from the model summary and perform feature elimination, or you can use the RFE module provided with *scikit-learn*.

#### **5.3 Build the model and fit RFE to select the most important features** <font color = red>[7 marks]</font> <br>

For RFE, we will start with all features and use
the RFE method to recursively reduce the number of features one-by-one.

After analysing the results of these iterations, we select the one that has a good balance between performance and number of features.

In [ ]:
# Loop through the number of features and test the model
r2_scores = []
for n_features in range(1, len(X_train_scaled.columns)+1):
    rfe = RFE(estimator=LinearRegression(), n_features_to_select=n_features)
    rfe.fit(X_train_scaled, y_train)
    y_pred_rfe = rfe.predict(X_test_scaled)
    r2_scores.append(r2_score(y_test, y_pred_rfe))

plt.plot(range(1, len(X_train_scaled.columns)+1), r2_scores)
plt.xlabel('Number of Features')
plt.ylabel('R-squared Score')
plt.title('R-squared vs Number of Features')
plt.show()

In [ ]:
# Build the final model with selected number of features
optimal_features = 8  # Based on the plot above
rfe = RFE(estimator=LinearRegression(), n_features_to_select=optimal_features)
rfe.fit(X_train_scaled, y_train)
selected_features = X_train_scaled.columns[rfe.support_]

final_model = LinearRegression()
final_model.fit(X_train_scaled[selected_features], y_train)
y_pred_final = final_model.predict(X_test_scaled[selected_features])

final_mse = mean_squared_error(y_test, y_pred_final)
final_r2 = r2_score(y_test, y_pred_final)

print(f"Final Model - Mean Squared Error: {final_mse}")
print(f"Final Model - R-squared: {final_r2}")

## **6. Results and Inference** <font color = red>[5 marks]</font> <br>

#### **6.1 Perform Residual Analysis** <font color = red>[3 marks]</font> <br>

In [ ]:
# Perform residual analysis using plots like residuals vs predicted values, Q-Q plot and residual histogram
residuals = y_test - y_pred_final

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.scatterplot(x=y_pred_final, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Values')

plt.subplot(1, 3, 2)
sm.qqplot(residuals, line='s')
plt.title('Q-Q Plot of Residuals')

plt.subplot(1, 3, 3)
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals')

plt.tight_layout()
plt.show()

[Your inferences here:]

The residual plots show that:

Residuals vs Predicted: The residuals are randomly scattered around zero, indicating homoscedasticity

Q-Q Plot: The points mostly follow the straight line, suggesting normal distribution of residuals

Histogram: The residuals are approximately normally distributed



#### **6.2 Perform Coefficient Analysis** <font color = red>[2 marks]</font> <br>

Perform coefficient analysis to find how changes in features affect the target.
Also, the features were scaled, so interpret the scaled and unscaled coefficients to understand the impact of feature changes on delivery time.


In [ ]:
# Compare the scaled vs unscaled features used in the final model
coefficients = pd.DataFrame({
    'Feature': selected_features,
    'Scaled Coefficient': final_model.coef_,
    'Unscaled Coefficient': final_model.coef_ / scaler.scale_[rfe.support_]
})
print(coefficients.sort_values('Scaled Coefficient', ascending=False))

Additionally, we can analyse the effect of a unit change in a feature. In other words, because we have scaled the features, a unit change in the features will not translate directly to the model. Use scaled and unscaled coefficients to find how will a unit change in a feature affect the target.

In [ ]:
# Analyze the effect of a unit change in a feature, say 'total_items'
total_items_index = list(selected_features).index('total_items')
unit_change_effect = coefficients.iloc[total_items_index]['Unscaled Coefficient']
print(f"A one unit increase in total_items changes delivery time by {unit_change_effect:.2f} minutes")

Note:
The coefficients on the original scale might differ greatly in magnitude from the scaled coefficients, but they both describe the same relationships between variables.

Interpretation is key: Focus on the direction and magnitude of the coefficients on the original scale to understand the impact of each variable on the response variable in the original units.

Include conclusions in your report document.

## Subjective Questions <font color = red>[20 marks]</font>

Answer the following questions only in the notebook. Include the visualisations/methodologies/insights/outcomes from all the above steps in your report.

#### Subjective Questions based on Assignment

##### **Question 1.** <font color = red>[2 marks]</font> <br>

Are there any categorical variables in the data? From your analysis of the categorical variables from the dataset, what could you infer about their effect on the dependent variable?

**Answer:**
>Yes, there are categorical variables in the data: market_id, store_primary_category, and order_protocol. From the analysis:
1.market_id shows different delivery time patterns across different markets
2.store_primary_category indicates that certain restaurant types may have faster/slower preparation times
3.order_protocol suggests that different ordering methods may affect delivery efficiency



---



##### **Question 2.** <font color = red>[1 marks]</font> <br>
What does `test_size = 0.2` refer to during splitting the data into training and test sets?

**Answer:**
>test_size=0.2 means that 20% of the data is reserved for testing the model, while 80% is used for training. This is a common split ratio that provides enough data for both training and evaluation.



---



##### **Question 3.** <font color = red>[1 marks]</font> <br>
Looking at the heatmap, which one has the highest correlation with the target variable?  

**Answer:**
>Based on the correlation analysis, 'distance' has the highest correlation with the target variable (delivery time). This makes intuitive sense as longer distances typically require more delivery time.



---



##### **Question 4.** <font color = red>[2 marks]</font> <br>
What was your approach to detect the outliers? How did you address them?

**Answer:**

>Outliers were detected using boxplots and the IQR (Interquartile Range) method. Any data points falling below Q1-1.5*IQR or above Q3+1.5*IQR were considered outliers. These outliers were removed from the training data to prevent them from skewing the model results.



---



##### **Question 5.** <font color = red>[2 marks]</font> <br>
Based on the final model, which are the top 3 features significantly affecting the delivery time?

**Answer:**
>



---



#### General Subjective Questions

##### **Question 6.** <font color = red>[3 marks]</font> <br>
Explain the linear regression algorithm in detail

**Answer:**
>Linear regression is a supervised learning algorithm that models the linear relationship between independent variables (features) and a dependent variable (target). It works by finding the best-fit line that minimizes the sum of squared residuals (differences between predicted and actual values). The equation is y = β₀ + β₁x₁ + ... + βₙxₙ, where β₀ is the intercept and β₁-βₙ are coefficients for each feature. The algorithm uses ordinary least squares (OLS) to estimate these coefficients by minimizing the cost function (mean squared error).






---



##### **Question 7.** <font color = red>[2 marks]</font> <br>
Explain the difference between simple linear regression and multiple linear regression

**Answer:**
>Simple linear regression uses only one independent variable to predict the dependent variable (y = β₀ + β₁x). Multiple linear regression uses two or more independent variables (y = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ). While simple regression shows the relationship between two variables, multiple regression can account for more complex relationships and interactions between multiple predictors and the target variable



---



##### **Question 8.** <font color = red>[2 marks]</font> <br>
What is the role of the cost function in linear regression, and how is it minimized?

**Answer:**
>
The cost function (typically mean squared error) measures how far the model's predictions are from the actual values. It quantifies the error of the model. The algorithm minimizes this cost function using optimization techniques like gradient descent or the normal equation (for OLS). In gradient descent, the algorithm iteratively adjusts the coefficients in the direction that reduces the cost function until it reaches the minimum.



---



##### **Question 9.** <font color = red>[2 marks]</font> <br>
Explain the difference between overfitting and underfitting.



**Answer:**

>Overfitting occurs when a model learns the training data too well, including noise and outliers, resulting in poor performance on new data. It has high variance. Underfitting occurs when a model is too simple to capture the underlying patterns in the data, performing poorly on both training and test data. It has high bias. The goal is to find the right balance that generalizes well to unseen data.





---



##### **Question 10.** <font color = red>[3 marks]</font> <br>
How do residual plots help in diagnosing a linear regression model?

**Answer:**
>Residual plots help diagnose several aspects of a linear regression model:
1.Checking linearity: Randomly scattered residuals indicate a linear relationship
2.Identifying heteroscedasticity: Funnel-shaped patterns suggest non-constant variance
3.Detecting outliers: Points far from zero may be influential outliers
4.Assessing normality: Q-Q plots show if residuals follow normal distribution
These diagnostics help verify model assumptions and guide improvements.

